In [8]:
from netCDF4 import Dataset
import zarr, h5py, fsspec
import xarray as xr
import os.path
from mpi4py import MPI



leucht: RTC timebase, using 213 picos/cycle from /proc instead of the detected 195 picos/cycle

leucht.26733RTC timebase, using 213 picos/cycle from /proc instead of the detected 195 picos/cycle



In [2]:
%%script false --no-raise-error
if __name__ == '__main__':
    #Todo read up on hdf5, zarr, netcdf, xarray, mpi, lustre, ceph
    # and check back with dkrz their structure, code conventions, datasets

    # Todo setup test cases proactively to ensure proper comparability and best practice

    #Todo use netcdf dataset for testing (for now) should already bet hdf5
    dest_hdf = Dataset("data/source.nc", "w", format="NETCDF4")
    print(dest_hdf.data_model)
    dest_hdf.close()

    #Todo import netcdf data through hdf5 to zarr (for now)
    dest_zarr = zarr.open_group('data/example2.zarr', mode='w')
    zarr.copy_all(dest_hdf, dest_zarr)
    dest_zarr.tree()

    #Todo conversion to netcdf

    #for hdf5 nothing needs to be done

    #for zarr (wip). netcdf is developing its own implementation
    # but that isn't available yet so I will work on something myself in the meantime

    #Todo setup Lustre and Ceph (need info on that)

    #Todo setup benchmark (prob compression, filesize, access-time, r/w-time) for sequential access

    #Todo setup benchmark for parallel access

    #Todo setup benchmark for random access

    #Todo setup benchmark for parallel with subfileing and async / I/O


In [6]:
from keras.datasets import cifar10
import numpy as np
import h5py

(x_img_train, y_label_train), (x_img_test, y_label_test) = cifar10.load_data()

# create HDF5 file
with h5py.File('data/test_ds.hdf5', 'w') as hf:
    u = hf.create_dataset("u", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)
    v = hf.create_dataset("v", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)
    w = hf.create_dataset("w", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)
    x = hf.create_dataset("x", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)
    y = hf.create_dataset("y", data=np.random.rand(1_000_000, 10, 10, 1), shape=(1_000_000, 10, 10, 1), compression="gzip", chunks=True)

hf.close()   

In [9]:
#Todo prepare dataset for testing

# filepath_zarr = 'https://power-datastore.s3.amazonaws.com/v9/climatology/power_901_climatology_meteorology_utc.zarr'
# 
# filepath_mapped_zarr = fsspec.get_mapper(filepath_zarr)

ds = xr.open_dataset("data/test_ds.hdf5", engine="h5netcdf", phony_dims="access")

ds.to_netcdf('data/test_dataset.nc', mode="w")

ds.to_zarr("data/test_dataset.zarr", mode="w")

filepath_mapped_zarr = 'data/test_dataset.zarr'

filepath_nc = 'data/test_dataset.nc'



In [26]:
%%time 
ds_zarr = xr.open_zarr(filepath_mapped_zarr, consolidated=True)


CPU times: user 1.88 ms, sys: 0 ns, total: 1.88 ms
Wall time: 1.38 ms


In [23]:
%%time 
max_var = ds_zarr["x"].max().compute()


CPU times: user 19 ms, sys: 0 ns, total: 19 ms
Wall time: 16.4 ms


In [20]:
%%time
ds_hdf5 = xr.open_dataset(filename_or_obj=filepath_nc, engine="h5netcdf")


CPU times: user 9.08 ms, sys: 0 ns, total: 9.08 ms
Wall time: 7.7 ms


In [16]:
%%time
max_var = ds_hdf5["x"].max().compute()

CPU times: user 7.05 ms, sys: 10.7 ms, total: 17.7 ms
Wall time: 15.5 ms


In [10]:
%%script false --no-raise-error
import xarray as xr
ds1 = xr.open_dataset(filename_or_obj="data/4f6e0dccfcfe4f76f1e64f187cc0551c.nc", engine="netcdf4")